In [19]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
import time
from collections import Counter
import pickle
from tqdm import tqdm

import matplotlib.pyplot as plt
import seaborn as sns

from matplotlib import font_manager, rc  
font_name = font_manager.FontProperties(fname='C:/Windows/Fonts/malgun.ttf').get_name()
rc('font', family=font_name)

##### fuleaf사이트 불러오기

In [20]:
driver = webdriver.Chrome(ChromeDriverManager().install())
#네이버 지식인: 식물키우기
driver.get("https://www.fuleaf.com/plants")
driver.maximize_window()
time.sleep(1)

##### 스크롤 내려서 더보기 버튼 다 누르기

In [21]:
for i in range(15):
    try :
        driver.implicitly_wait(20)        
        #더보기버튼 
        driver.find_element_by_xpath('//*[@id="more_btn"]').send_keys(Keys.ENTER)
    except :
        pass

##### href: url 주소 저장하기

In [22]:
#객체화
soup = bs(driver.page_source, 'html.parser')
url = soup.select('div.plants__list-item>a')
url_list=[]
for i in url:
    #print('https://www.fuleaf.com/'+i['href'])
    url_list.append('https://www.fuleaf.com/'+i['href'])

In [23]:
len(url_list)

306

In [24]:
title_list=[]
temp_list=[]                             
water_list=[]
humidity_list=[]
for url in tqdm(url_list):
    driver.get(url)
    time.sleep(1)
    soup = bs(driver.page_source, 'html.parser')


    title=driver.find_elements_by_xpath('//*[@id="plantDetail__page"]/div/section/div[2]/section[2]/div[1]/div/h2')                                                                                
    for j in title:
        title_list.append(j.text)
    
    temp=driver.find_elements_by_css_selector('#plantDetail__page > div > section > div.pcSlide__flex.layout_body > section.plants-simpleinfo-section.layout_body > div.simpleinfo-table > ul > div:nth-child(2) > li:nth-child(2) > h2')
    for j in temp:                              
        temp_list.append(j.text)

    water = driver.find_elements_by_css_selector('#plantDetail__page > div > section > div.pcSlide__flex.layout_body > section.plants-simpleinfo-section.layout_body > div.simpleinfo-table > ul > div:nth-child(1) > li:nth-child(1) > h1')
    for j in water:                              
        water_list.append(j.text)
    time.sleep(1)

    humidity = driver.find_elements_by_css_selector('#plantDetail__page > div > section > div.pcSlide__flex.layout_body > section.plants-simpleinfo-section.layout_body > div.simpleinfo-table > ul > div:nth-child(2) > li:nth-child(1) > h1')
    for j in humidity:                              
        humidity_list.append(j.text)

    #새창불러오기
    #driver.get("https://www.fuleaf.com/plants")

100%|██████████| 306/306 [3:33:22<00:00, 41.84s/it]  


In [32]:
len(title_list), len(temp_list), len(water_list), len(humidity_list)

(162, 162, 162, 162)

##### 텍스트 정제-temp_list

In [33]:
from konlpy.tag import Okt
okt=Okt()
import re

#필요없는 표시들 지우기
temp2_list =[]
for i in temp_list:
    result = i.replace("\n", "")
    #        정규표현식
    result = re.compile("의.+").sub("", result)
    #result = result.replace("℃", "")
    #result = result.replace("°C", "")
    result = result.replace("-", "~")

    temp2_list.append(result)

##### DataFrame

In [34]:
data = {'이름' : title_list, '온도' : temp2_list, '물' : water_list, '습도': humidity_list}
plants = pd.DataFrame(data)
plants

,이름,온도,물,습도
0,바나나 크로톤,21~27℃,평균 주 1~2회,40~70%
1,코니오그램 에메이엔시스 '골든 제브라',21~25℃,평균 주 1~2회,70% 이상
2,베고니아 콘치폴리아,21~25℃,평균 주 1~2회,40~70%
3,싱고니움 밀크컨페티,21~25℃,평균 주 1~2회,40~70%
4,필로덴드론 옥시카르디움 '브라질',16~26℃,평균 주 1~2회,40~70%
...,...,...,...,...
157,디지고테카 아랄리아,15~25℃,평균 주 1~2회,40~70%
158,드라세나 마지나타,24~32℃,평균 월 1~2회,40~70%
159,덕구리 난,21~25°C,평균 월 1~2회,40% 이하
160,금전수 (돈나무),16~20℃,평균 월 1~2회,40~70%


##### 레벨나누기-중심은 물주는 주기
- lv.1: 평균 월 1회 이하 + 습도 40%이하, 40~70%
- lv.2: 평균 월 1~2회 + 습도 40%이하, 40~70%
- lv.3: 평균 주 1~2회 + 습도 40~70%
- lv.4: 평균 주 1~2회 + 습도 70% 이상
- lv.5: 평균 주 2회 이상 + 습도 습도 40~70%
- lv.6: 평균 주 2회 이상 + 습도 70% 이상

In [35]:
plants.loc[plants.물 == '평균 월 1회 이하', 'level'] =1
plants.loc[plants.물 == '평균 월 1~2회', 'level'] =2
plants.loc[(plants.물 == '평균 주 1~2회') & (plants.습도 == '40~70%'), 'level'] =3
plants.loc[(plants.물 == '평균 주 1~2회') & (plants.습도 == '70% 이상'), 'level'] =4
plants.loc[(plants.물 == '평균 주 2회 이상') & (plants.습도 == '40~70%'), 'level'] =5
plants.loc[(plants.물 == '평균 주 2회 이상') & (plants.습도 == '70% 이상'), 'level'] =6

In [36]:
plants

,이름,온도,물,습도,level
0,바나나 크로톤,21~27℃,평균 주 1~2회,40~70%,3.0
1,코니오그램 에메이엔시스 '골든 제브라',21~25℃,평균 주 1~2회,70% 이상,4.0
2,베고니아 콘치폴리아,21~25℃,평균 주 1~2회,40~70%,3.0
3,싱고니움 밀크컨페티,21~25℃,평균 주 1~2회,40~70%,3.0
4,필로덴드론 옥시카르디움 '브라질',16~26℃,평균 주 1~2회,40~70%,3.0
...,...,...,...,...,...
157,디지고테카 아랄리아,15~25℃,평균 주 1~2회,40~70%,3.0
158,드라세나 마지나타,24~32℃,평균 월 1~2회,40~70%,2.0
159,덕구리 난,21~25°C,평균 월 1~2회,40% 이하,2.0
160,금전수 (돈나무),16~20℃,평균 월 1~2회,40~70%,2.0


##### columns level: 데이터 타입 변경해주기

In [39]:
plants['level']=plants.level.astype(int)
plants

,이름,온도,물,습도,level
0,바나나 크로톤,21~27℃,평균 주 1~2회,40~70%,3
1,코니오그램 에메이엔시스 '골든 제브라',21~25℃,평균 주 1~2회,70% 이상,4
2,베고니아 콘치폴리아,21~25℃,평균 주 1~2회,40~70%,3
3,싱고니움 밀크컨페티,21~25℃,평균 주 1~2회,40~70%,3
4,필로덴드론 옥시카르디움 '브라질',16~26℃,평균 주 1~2회,40~70%,3
...,...,...,...,...,...
157,디지고테카 아랄리아,15~25℃,평균 주 1~2회,40~70%,3
158,드라세나 마지나타,24~32℃,평균 월 1~2회,40~70%,2
159,덕구리 난,21~25°C,평균 월 1~2회,40% 이하,2
160,금전수 (돈나무),16~20℃,평균 월 1~2회,40~70%,2


##### Plants CSV파일로 저장하기

In [40]:
plants.to_csv('Plants.csv', encoding='utf-8-sig')